In [1]:
import cv2
import numpy as np

# Function to extract ROIs with blinking lights
def extract_blinking_lights(image_path):
    # Load the image
    img = cv2.imread(image_path)

    if img is None:
        print(f"Error: Unable to load image from {image_path}")
    else:
        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Apply thresholding to highlight the lights
        _, thresholded = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

        
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Extract ROIs based on contours
        rois = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            w = w + 10
            h = h + 10
            x = x - 5
            y = y - 5
            print(f'x: {x}, y: {y}, w: {w}, h: {h}')
            roi = img[y:y+h, x:x+w]
            rois.append(roi)
        
        return rois

# Example usage
image_path_newlights = 'labeled_frames_test\\test.mp4_00000_2.jpg'
image_path_blacklights = 'labeled_frames_fixed\\vid9.mp4_00001_4.jpg'
image_path_oldlights = 'labeled_frames_test\\vid7.mp4_00000_4.jpg'
rois = extract_blinking_lights(image_path_blacklights)

# Display the original image and extracted ROIs
cv2.imshow('Original Image', cv2.imread(image_path_blacklights))
light_count = len(rois)
for i, roi in enumerate(rois):
    cv2.imshow(f'ROI {i+1}', roi)
print(f'Number of blinking lights: {light_count}')
cv2.waitKey(0)
cv2.destroyAllWindows()

x: 509, y: 829, w: 13, h: 12
x: 1381, y: 781, w: 15, h: 13
x: 463, y: 287, w: 17, h: 16
x: 1408, y: 285, w: 13, h: 14
Number of blinking lights: 4


In [2]:
from glob import glob
import os

framelist = []
data_folder = 'labeled_frames_fixed'

for image_path in glob(os.path.join(data_folder, '*.jpg')):
    framelist.append(image_path)

# framelist = [
#              'labeled_frames_fixed\\vid9.mp4_00054_2.jpg', 
#              'labeled_frames_fixed\\vid9.mp4_00047_0.jpg', 
#              'labeled_frames_fixed\\vid9.mp4_00053_3.jpg', 
#              'labeled_frames_fixed\\vid9.mp4_00046_2.jpg',
#              ]



In [6]:
import cv2
import numpy as np

def extract_blinking_lights(image_path):
    img = cv2.imread(image_path)

    if img is None:
        print(f"Error: Unable to load image from {image_path}")
    else:
        # Convert the image to grayscale
        grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Apply thresholding to highlight the lights
        _, thresholded = cv2.threshold(grey, 200, 255, cv2.THRESH_BINARY)
        
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Extract ROIs based on contours
        rois = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            w = w + 20
            h = h + 20
            x = x - 10
            y = y - 10
            
            rois.append((x,y,w,h))
        
        return rois

def are_similar_coordinates(coord1, coord2):
    x1, y1, w1, h1 = coord1
    x2, y2, w2, h2 = coord2
    
    # Adjust these threshold values based on your requirements
    position_threshold = 50  # Adjust as needed
    size_threshold = 50  # Adjust as needed
    
    # Check if the coordinates are similar in terms of position and size
    position_diff = abs(x1 - x2) + abs(y1 - y2)
    size_diff = abs(w1 - w2) + abs(h1 - h2)
    
    return position_diff < position_threshold and size_diff < size_threshold

# Example usage
merged_rois = []
all_rois = []

for image_path in framelist:
    rois = extract_blinking_lights(image_path)
    for roi in rois:
        all_rois.append(roi)
        

if all_rois is not None:
    for roi_coordinates in all_rois:
        add_to_merged = True

        for (merged_roi, merged_roi_coordinates) in merged_rois:
            if are_similar_coordinates(roi_coordinates, merged_roi_coordinates):
                add_to_merged = False
                break
        
        if add_to_merged:
            merged_rois.append((roi, roi_coordinates))
            
print(f'Number of blinking lights: {len(merged_rois)}')

#print all coordinates
for i, (roi, coordinates) in enumerate(merged_rois):
    print(coordinates)
    #use coordinates to create image roi
    x, y, w, h = coordinates
    #extract from original image
    img = cv2.imread(framelist[0])
    roi = img[y:y+h, x:x+w]
    #display image
    cv2.imshow(f'ROI {i}', roi)

cv2.waitKey(0)
cv2.destroyAllWindows()
    


Number of blinking lights: 4
(1374, 773, 27, 27)
(458, 283, 27, 27)
(1403, 280, 23, 24)
(504, 824, 23, 22)


In [ ]:
import cv2
import numpy as np

# Open the video file
video_path = './vidscapstone/vid9.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

rois = merged_rois

# Loop through frames
while True:
    print('________________________________________________________')
    # Read a frame
    ret, frame = cap.read()

    # Break the loop if the video is finished
    if not ret:
        break

    count = 0

    for _, roi in rois:
        x, y, w, h = roi
        roi_processed = frame[y:y+h, x:x+w]
        roi_processed = cv2.resize(roi_processed, (50, 50))
        roi_processed = cv2.convertScaleAbs(roi_processed)
        roi_processed = np.array(roi_processed)
        roi_processed = roi_processed / 255.0
        roi_processed = np.expand_dims(roi_processed, axis=0)
        pred = model.predict(roi_processed)
    #     print(pred)
    #     if pred > 0.5:
    #         count += 1

    #     cv2.imshow(f'ROI {x}, {y}', frame[y:y+h, x:x+w])

    #     breaker = False
    #     # Press q to exit
    #     if cv2.waitKey(0) & 0xFF == ord('q'):
    #         cv2.destroyAllWindows()
    #         breaker = True
    #         break
    #     else:
    #         cv2.destroyAllWindows()

    # if breaker:
    #     break


    #frame_to_show = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    #cv2.imshow(f'Turned on lights {count}', frame_to_show)
    
    # print(f'Turned on lights: {count}')

# Release the video capture object
cap.release()

# Close all OpenCV windows
cv2.destroyAllWindows()
